Imports

In [2]:
import pymysql
import pandas as pd
from config import DATABASE_CONFIG

Create a database and connect to it

In [3]:
def create_database():
    # Connect to the database    
    connection = pymysql.connect(**DATABASE_CONFIG)
    
    cursor = connection.cursor()

    database = "first_project"

    # If the database exists, drop it
    cursor.execute(f"DROP DATABASE IF EXISTS {database}")
    
    # Create the database
    cursor.execute(f"CREATE DATABASE {database}")
    print(f"Database {database} created successfully")

    # Connect to the new database
    connection = pymysql.connect(**DATABASE_CONFIG, database=database)

    cursor = connection.cursor()

    return cursor, connection

Open the files and create dataframes

In [4]:
agents = pd.read_csv("agents.csv")
agents.head()

,agentid,name
0,0,Michele Williams
1,1,Jocelyn Parker
2,2,Christopher Moreno
3,3,Todd Morrow
4,4,Randy Moore


In [5]:
calls = pd.read_csv("calls.csv")
calls.head()

,callid,agentid,customerid,pickedup,duration,productsold
0,0,10,179,0,0,0
1,1,5,691,1,116,0
2,2,10,80,1,165,0
3,3,6,629,1,128,0
4,4,8,318,1,205,0


In [6]:
customers = pd.read_csv("customers.csv")
# Keep just the necessary columns
customers = customers[["customerid", "name", "occupation", "Age"]]
customers.head()

,customerid,name,occupation,Age
0,0,David Melton,Unemployed,16
1,1,Michael Gonzalez,Student,19
2,2,Amanda Wilson,Student,18
3,3,Robert Thomas,"Engineer, structural",25
4,4,Eddie Hall,Surgeon,30


Create the cursor and the connection

In [7]:
cursor, connection = create_database()

Database first_project created successfully


SQL CREATE sentences

In [8]:
calls_create = (""" CREATE TABLE IF NOT EXISTS calls (
                callid INT PRIMARY KEY,
                agentid INT,
                customerid INT, 
                pickedup INT,
                duration INT,
                productsold INT
) """)

agents_create = (""" CREATE TABLE IF NOT EXISTS agents (
                agentid INT PRIMARY KEY,
                name VARCHAR(100)
) """)

customers_create = (""" CREATE TABLE IF NOT EXISTS customer (
                customerid INT PRIMARY KEY, 
                name VARCHAR(100),
                occupation VARCHAR(255),
                age INT
) """)

cursor.execute(calls_create)
cursor.execute(agents_create)
cursor.execute(customers_create)

connection.commit()

Basic data insertion

In [9]:
calls_insert = (""" INSERT INTO calls (callid, agentid, customerid, pickedup, duration, productsold) 
                VALUES (%s, %s, %s, %s, %s, %s) """)

agents_insert = (""" INSERT INTO agents (agentid, name)
                 VALUES (%s, %s)""")

customers_insert = ("""INSERT INTO customer (customerid, name, occupation, age) 
                    VALUES (%s, %s, %s, %s)""")

for i, row in calls.iterrows():
    cursor.execute(calls_insert, list(row))

for i, row in agents.iterrows():
    cursor.execute(agents_insert, list(row))

for i, row in customers.iterrows():
    cursor.execute(customers_insert, list(row))

connection.commit()

Close cursor and connection

In [10]:
if 'cursor' in locals():
    cursor.close()
if 'connection' in locals() and connection.open:
    connection.close()